In [1]:
from __future__ import division
import ROOT
from ROOT import gSystem, gInterpreter
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
import math
import gc
import itertools

import os
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(linewidth=np.inf)

Welcome to JupyROOT 6.20/04


In [3]:
gInterpreter.AddIncludePath("/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/")
gInterpreter.AddIncludePath("/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/external/")
gInterpreter.Declare('#include "/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/classes/DelphesClasses.h"')
gInterpreter.Declare('#include "/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/external/ExRootAnalysis/ExRootAnalysisLinkDef.h"')
gSystem.Load("/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/libDelphes.so")

1

In [4]:
from ROOT import Tower, Track

In [5]:
Signal = ROOT.TChain("Delphes")
# Z_plus_jets_1 = ROOT.TChain("Delphes")
# Z_plus_jets_2 = ROOT.TChain("Delphes")
# Z_plus_jets_3 = ROOT.TChain("Delphes")
# Z_plus_jets_4 = ROOT.TChain("Delphes")
# Z_plus_jets_5 = ROOT.TChain("Delphes")
# Z_plus_jets_6 = ROOT.TChain("Delphes")
# Z_plus_jets_7 = ROOT.TChain("Delphes")
# Z_plus_jets_8 = ROOT.TChain("Delphes")
# Z_plus_jets_9 = ROOT.TChain("Delphes")
# Z_plus_jets_10 = ROOT.TChain("Delphes")


# chain_list = [Z_plus_jets_1, Z_plus_jets_2, Z_plus_jets_3, Z_plus_jets_4, Z_plus_jets_5,
#               Z_plus_jets_6, Z_plus_jets_7, Z_plus_jets_8, Z_plus_jets_9, Z_plus_jets_10,
#               Signal]

chain_list = [Signal]

# Events = {'Z_plus_jets_1':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_1.root',
#           'Z_plus_jets_2':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_2.root',
#           'Z_plus_jets_3':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_3.root',
#           'Z_plus_jets_4':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_4.root',
#           'Z_plus_jets_5':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_5.root',
#           'Z_plus_jets_6':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_6.root',
#           'Z_plus_jets_7':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_7.root',
#           'Z_plus_jets_8':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_8.root',
#           'Z_plus_jets_9':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_9.root',
#           'Z_plus_jets_10':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/Z_plus_jets_10.root',
#           'Signal':'/home/felipe/JoaoPino/1-BGL-MLProject/roots/314p99GeV.root'}

Events = {'Signal': '/home/felipe/JoaoPino/1-BGL-MLProject/roots/314p99GeV.root'}

for i,j in enumerate(Events.items()):
    chain_list[i].Add(j[1])
    print('number of events {}:'.format(j[0]),chain_list[i].GetEntries())

number of events Signal: 100000


# Code with Jet matching

In [6]:
Features = ['M_A2_1', 'M_A2_2', 'M_H', 'pt_l1', 'pt_l2', 'pt_jet1', 'pt_jet2', 'pt_jet3', 'pt_jet4', 'E_l1',
            'E_l2', 'E_j1', 'E_j2', 'E_j3', 'E_j4', 'eta_l1', 'eta_l2', 'eta_j1', 'eta_j2', 'eta_j3', 'eta_j4', 'phi_l1',
            'phi_l2', 'phi_j1', 'phi_j2', 'phi_j3', 'phi_j4', 'MZ', 'pt_z', 'eta_z', 'phi_z', 'pt_A2_1', 'pt_A2_2', 'eta_A2_1', 'MH',
            'pt_H', 'eta_H', 'phi_H','eta_A2_2', 'phi_A2_1', 'phi_A2_2', 'MA3', 'pt_A3', 'eta_A3', 'phi_A3', 'cos_j1j2', 'cos_j1j3',
            'cos_j1j4', 'cos_j2j3','cos_j2j4', 'cos_j3j4', 'cos_j1lep1', 'cos_j1lep2', 'cos_j2lep1', 'cos_j2lep2', 'cos_j3lep1', 'cos_j3lep2', 'cos_j4lep1',
            'cos_j4lep2', 'cos_lep1lep2', 'DR_j1j2', 'DR_j1j3', 'DR_j1j4', 'DR_j2j3', 'DR_j2j4', 'DR_j3j4', 'DR_j1lep1', 'DR_j1lep2',
            'DR_j2lep1', 'DR_j2lep2', 'DR_j3lep1', 'DR_j3lep2', 'DR_j4lep1', 'DR_j4lep2', 'DR_lep1lep2', 'DPhi_j1j2', 'DPhi_j1j3',
            'DPhi_j1j4', 'DPhi_j2j3', 'DPhi_j2j4', 'DPhi_j3j4', 'DPhi_j1lep1', 'DPhi_j1lep2', 'DPhi_j2lep1', 'DPhi_j2lep2', 'DPhi_j3lep1',
            'DPhi_j3lep2', 'DPhi_j4lep1', 'DPhi_j4lep2', 'DPhi_lep1lep2', 'DTheta_j1j2', 'DTheta_j1j3', 'DTheta_j1j4', 'DTheta_j2j3',
            'DTheta_j2j4', 'DTheta_j3j4', 'DTheta_j1lep1', 'DTheta_j1lep2', 'DTheta_j2lep1', 'DTheta_j2lep2', 'DTheta_j3lep1',
            'DTheta_j3lep2', 'DTheta_j4lep1', 'DTheta_j4lep2', 'DTheta_lep1lep2', 'xs_weight',
            'DPhi_A2A2', 'DPhi_A2H2_1', 'DPhi_A2H2_2', 'DPhi_A2Z0_1', 'DPhi_A2Z0_1', 'DPhi_H2Z0', 'DR_A2A2', 'DR_A2H2_1', 'DR_A2H2_2',
            'DR_A2Z0_1', 'DR_A2Z0_1', 'DR_H2Z0', 'DTheta_A2A2', 'DTheta_A2H2_1', 'DTheta_A2H2_2', 'DTheta_A2Z0_1', 'DTheta_A2Z0_1',
            'DTheta_H2Z0', 'cos_A2A2', 'cos_A2H2_1', 'cos_A2H2_2', 'cos_A2Z0_1', 'cos_A2Z0_2', 'cos_H2Z0_2']

Mass_difference = 25

for chain, proc in zip(chain_list,Events.items()):
    
    df = pd.DataFrame(columns=Features)

    for count, event in enumerate(tqdm(chain, total=chain.GetEntries())):
        
        out = np.empty((chain.GetEntries(),len(Features)))
        out.fill(np.nan)
        survived = 0
        
        #if count == 100:
        #    break
            
        jet_eta = list(filter(lambda x: abs(x.Eta) <= 2.5, event.Jet))
        jet_filt = list(filter(lambda x: x.PT>20., jet_eta))
        jet_M = list(filter(lambda x: x.Mass>15., jet_filt))
        jet_filt_ord = sorted(jet_M, key=lambda Jet:Jet.PT, reverse=True)
        list_jets = jet_filt_ord
        
        #Get charged leptons (anti-muons and muons)
        muons_eta = list(filter(lambda Muons: np.abs(Muons.Eta) <= 2.47, event.Muon))
        muons = list(filter(lambda Muons: Muons.PT > 20, muons_eta))
        muons = sorted(muons, key=lambda Muon: Muon.PT, reverse=True)          
        
        list_muon = muons

        #Get charged leptons (anti-electrons and electrons)
        electrons_eta = list(filter(lambda Electrons: np.abs(Electrons.Eta) <= 2.47, event.Electron))
        electrons = list(filter(lambda Electrons: Electrons.PT > 20., electrons_eta))
        electrons = sorted(electrons, key=lambda Electron: Electron.PT, reverse=True)  

        list_electron = electrons
        
        list_charged_lep = electrons + muons

                    
        #Jet pairing
        if (len(list_jets) == 4 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())

            
            
            JET_LIST = [JET_1, JET_2, JET_3, JET_4]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)
            
        if (len(list_jets) == 5 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)
            
        if (len(list_jets) == 6 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)
        
        if (len(list_jets) == 7 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}

                        Dict_6.append(Dict)
            
        if (len(list_jets) == 8 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)
            
        if (len(list_jets) == 9 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)

  
        if (len(list_jets) == 10 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            JET_10 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_10.SetPtEtaPhiE(list_jets[9].P4().Pt(),list_jets[9].Eta,list_jets[9].Phi,list_jets[9].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9, JET_10]
                
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)
        
        
        if (len(list_jets) == 11 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            JET_10 = ROOT.TLorentzVector()
            JET_11 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_10.SetPtEtaPhiE(list_jets[9].P4().Pt(),list_jets[9].Eta,list_jets[9].Phi,list_jets[9].P4().E())
            JET_11.SetPtEtaPhiE(list_jets[10].P4().Pt(),list_jets[10].Eta,list_jets[10].Phi,list_jets[10].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9, JET_10, JET_11]
                
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)
                        
        if (len(list_jets) == 12 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            JET_10 = ROOT.TLorentzVector()
            JET_11 = ROOT.TLorentzVector()
            JET_12 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_10.SetPtEtaPhiE(list_jets[9].P4().Pt(),list_jets[9].Eta,list_jets[9].Phi,list_jets[9].P4().E())
            JET_11.SetPtEtaPhiE(list_jets[10].P4().Pt(),list_jets[10].Eta,list_jets[10].Phi,list_jets[10].P4().E())
            JET_12.SetPtEtaPhiE(list_jets[11].P4().Pt(),list_jets[11].Eta,list_jets[11].Phi,list_jets[11].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9, JET_10, JET_11, JET_12]
                
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)                        

        if (len(list_jets) == 13 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            JET_10 = ROOT.TLorentzVector()
            JET_11 = ROOT.TLorentzVector()
            JET_12 = ROOT.TLorentzVector()
            JET_13 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_10.SetPtEtaPhiE(list_jets[9].P4().Pt(),list_jets[9].Eta,list_jets[9].Phi,list_jets[9].P4().E())
            JET_11.SetPtEtaPhiE(list_jets[10].P4().Pt(),list_jets[10].Eta,list_jets[10].Phi,list_jets[10].P4().E())
            JET_12.SetPtEtaPhiE(list_jets[11].P4().Pt(),list_jets[11].Eta,list_jets[11].Phi,list_jets[11].P4().E())
            JET_13.SetPtEtaPhiE(list_jets[12].P4().Pt(),list_jets[12].Eta,list_jets[12].Phi,list_jets[12].P4().E())
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9, JET_10, JET_11, JET_12, JET_13]
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict) 

        if (len(list_jets) == 14 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            JET_10 = ROOT.TLorentzVector()
            JET_11 = ROOT.TLorentzVector()
            JET_12 = ROOT.TLorentzVector()
            JET_13 = ROOT.TLorentzVector()
            JET_14 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_10.SetPtEtaPhiE(list_jets[9].P4().Pt(),list_jets[9].Eta,list_jets[9].Phi,list_jets[9].P4().E())
            JET_11.SetPtEtaPhiE(list_jets[10].P4().Pt(),list_jets[10].Eta,list_jets[10].Phi,list_jets[10].P4().E())
            JET_12.SetPtEtaPhiE(list_jets[11].P4().Pt(),list_jets[11].Eta,list_jets[11].Phi,list_jets[11].P4().E())
            JET_13.SetPtEtaPhiE(list_jets[12].P4().Pt(),list_jets[12].Eta,list_jets[12].Phi,list_jets[12].P4().E())
            JET_14.SetPtEtaPhiE(list_jets[13].P4().Pt(),list_jets[13].Eta,list_jets[13].Phi,list_jets[13].P4().E())            
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9, JET_10, JET_11, JET_12, JET_13, JET_14]                        
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)                        
                        
        if (len(list_jets) == 15 and len(list_charged_lep) > 1):
            Dict_6 = []
            JET_1 = ROOT.TLorentzVector()
            JET_2 = ROOT.TLorentzVector()
            JET_3 = ROOT.TLorentzVector()
            JET_4 = ROOT.TLorentzVector()
            JET_5 = ROOT.TLorentzVector()
            JET_6 = ROOT.TLorentzVector()
            JET_7 = ROOT.TLorentzVector()
            JET_8 = ROOT.TLorentzVector()
            JET_9 = ROOT.TLorentzVector()
            JET_10 = ROOT.TLorentzVector()
            JET_11 = ROOT.TLorentzVector()
            JET_12 = ROOT.TLorentzVector()
            JET_13 = ROOT.TLorentzVector()
            JET_14 = ROOT.TLorentzVector()
            JET_15 = ROOT.TLorentzVector()
            LEP_1 = ROOT.TLorentzVector()
            LEP_2 = ROOT.TLorentzVector()
            LEP_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            LEP_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,list_charged_lep[1].Phi,list_charged_lep[1].P4().E())            
            JET_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,list_jets[0].Phi,list_jets[0].P4().E())
            JET_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,list_jets[1].Phi,list_jets[1].P4().E())
            JET_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,list_jets[2].Phi,list_jets[2].P4().E())
            JET_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,list_jets[3].Phi,list_jets[3].P4().E())
            JET_5.SetPtEtaPhiE(list_jets[4].P4().Pt(),list_jets[4].Eta,list_jets[4].Phi,list_jets[4].P4().E())
            JET_6.SetPtEtaPhiE(list_jets[5].P4().Pt(),list_jets[5].Eta,list_jets[5].Phi,list_jets[5].P4().E())
            JET_7.SetPtEtaPhiE(list_jets[6].P4().Pt(),list_jets[6].Eta,list_jets[6].Phi,list_jets[6].P4().E())
            JET_8.SetPtEtaPhiE(list_jets[7].P4().Pt(),list_jets[7].Eta,list_jets[7].Phi,list_jets[7].P4().E())
            JET_9.SetPtEtaPhiE(list_jets[8].P4().Pt(),list_jets[8].Eta,list_jets[8].Phi,list_jets[8].P4().E())
            JET_10.SetPtEtaPhiE(list_jets[9].P4().Pt(),list_jets[9].Eta,list_jets[9].Phi,list_jets[9].P4().E())
            JET_11.SetPtEtaPhiE(list_jets[10].P4().Pt(),list_jets[10].Eta,list_jets[10].Phi,list_jets[10].P4().E())
            JET_12.SetPtEtaPhiE(list_jets[11].P4().Pt(),list_jets[11].Eta,list_jets[11].Phi,list_jets[11].P4().E())
            JET_13.SetPtEtaPhiE(list_jets[12].P4().Pt(),list_jets[12].Eta,list_jets[12].Phi,list_jets[12].P4().E())
            JET_14.SetPtEtaPhiE(list_jets[13].P4().Pt(),list_jets[13].Eta,list_jets[13].Phi,list_jets[13].P4().E())   
            JET_15.SetPtEtaPhiE(list_jets[13].P4().Pt(),list_jets[13].Eta,list_jets[13].Phi,list_jets[13].P4().E())            
            JET_LIST = [JET_1, JET_2, JET_3, JET_4, JET_5, JET_6, JET_7, JET_8, JET_9, JET_10, JET_11, JET_12, JET_13, JET_14, JET_15]                        
            
            for jet in range(0, len(JET_LIST)+1):
                for subset in itertools.combinations(JET_LIST, jet):
                    
                    All_combinations = subset
                    
                    if len(All_combinations) == 2:
                        
                        list_twojets = list(All_combinations)
                        Dict = {"Mass_twojet": (list_twojets[0] + list_twojets[1]).M(),
                                "First_jet": list_twojets[0],
                                "Second_jet": list_twojets[1]}
                        
                        Dict_6.append(Dict)   
                        
                        
        if len(list_jets) > 15:
            continue
            
        if len(list_jets) < 4:
            continue
            
        if len(list_charged_lep) < 2:
            continue
        
        Delta_Mass_dict = {"Delta_mass_twojets":[], "Jet1":[], "Jet2":[], "Jet3":[], "Jet4":[]}
        Mass_matrix = np.zeros([len(Dict_6),len(Dict_6)])
        for i in range(0,len(Dict_6)):
            for j in range(0,len(Dict_6)):
                Delta_Mass = Dict_6[i]["Mass_twojet"] - Dict_6[j]["Mass_twojet"]
                Jet1 = Dict_6[i]["First_jet"]
                Jet2 = Dict_6[i]["Second_jet"]
                Jet3 = Dict_6[j]["First_jet"]
                Jet4 = Dict_6[j]["Second_jet"]
                Delta_Mass_dict["Delta_mass_twojets"].append(Delta_Mass)
                Delta_Mass_dict["Jet1"].append(Jet1)
                Delta_Mass_dict["Jet2"].append(Jet2)
                Delta_Mass_dict["Jet3"].append(Jet3)
                Delta_Mass_dict["Jet4"].append(Jet4)

                Mass_matrix[i,j] = Delta_Mass

        #First check
        np.fill_diagonal(Mass_matrix, 1e10)
        if np.min(np.abs((Mass_matrix))) > Mass_difference:
            continue  
        np.fill_diagonal(Mass_matrix, 0)

        #Only combinations on the anti-diagonal are relevant
        Anti_diagonal = np.fliplr(Mass_matrix).diagonal()
        Lista_diagonal = list(np.abs(Anti_diagonal))
        Get_min = min([value for value in Lista_diagonal if value!=0])
        Index = np.where(Mass_matrix==Get_min)
        

        Get_jets = np.where(Delta_Mass_dict['Delta_mass_twojets']==Mass_matrix[Index[0][0], Index[1][0]])[0][0]

        Mass_delta = Delta_Mass_dict['Delta_mass_twojets'][Get_jets]
        Jet1_4vector = Delta_Mass_dict['Jet1'][Get_jets]
        Jet2_4vector = Delta_Mass_dict['Jet2'][Get_jets]
        Jet3_4vector = Delta_Mass_dict['Jet3'][Get_jets]
        Jet4_4vector = Delta_Mass_dict['Jet4'][Get_jets]

        #Second check
        if Mass_delta > Mass_difference:
            continue  
    
        if (Jet1_4vector == Jet2_4vector):
            continue
       
        if (Jet1_4vector == Jet3_4vector):
                continue
       
        if (Jet1_4vector == Jet4_4vector):
                continue
                
        if (Jet2_4vector == Jet3_4vector):
                continue   
                
        if (Jet2_4vector == Jet4_4vector):
                continue                 

        if (Jet3_4vector == Jet4_4vector):
                continue   
                
        M_A2_1 = (Jet1_4vector + Jet2_4vector).M()
        M_A2_2 = (Jet3_4vector + Jet4_4vector).M()
        M_j1j3 = (Jet1_4vector + Jet3_4vector).M()
        M_j2j4 = (Jet2_4vector + Jet4_4vector).M()
        M_j2j3 = (Jet2_4vector + Jet4_4vector).M()

        
        Mass_teste1 = (Jet1_4vector + Jet2_4vector + LEP_1 + LEP_2).M()
        Mass_teste2 = (Jet3_4vector + Jet4_4vector + LEP_1 + LEP_2).M()
        
        array = np.array([Mass_teste1, Mass_teste2])
        idx = (np.abs(array - 600)).argmin()
        MH = array[idx]

        if idx == 0:
            H2_vec = Jet1_4vector + Jet2_4vector + LEP_1 + LEP_2
            A2_H_vec = Jet1_4vector + Jet2_4vector
            A2_NotH_vec = Jet3_4vector + Jet4_4vector
            pt_H = H2_vec.Pt()
            eta_H = H2_vec.Eta()
            phi_H = H2_vec.Phi()
        
        if idx == 1:
            H2_vec = Jet3_4vector + Jet4_4vector + LEP_1 + LEP_2
            A2_H_vec = Jet3_4vector + Jet4_4vector
            A2_NotH_vec = Jet1_4vector + Jet2_4vector
            pt_H = H2_vec.Pt()
            eta_H = H2_vec.Eta()
            phi_H = H2_vec.Phi()

        #High-level observables from final states 

        # ------- Transverse momentum -------
        pt_l1 = LEP_1.Pt()
        pt_l2 = LEP_2.Pt()
        pt_jet1 = Jet1_4vector.Pt()
        pt_jet2 = Jet2_4vector.Pt()
        pt_jet3 = Jet3_4vector.Pt()
        pt_jet4 = Jet4_4vector.Pt()


        # ------- Energy -------
        E_l1 = LEP_1.E()
        E_l2 = LEP_2.E()
        E_j1 = Jet1_4vector.E()
        E_j2 = Jet2_4vector.E()            
        E_j3 = Jet3_4vector.E()
        E_j4 = Jet4_4vector.E() 

        # ------- Pseudorapidity -------
        eta_l1 = LEP_1.Eta()
        eta_l2 = LEP_2.Eta()
        eta_j1 = Jet1_4vector.Eta()
        eta_j2 = Jet2_4vector.Eta()          
        eta_j3 = Jet3_4vector.Eta()
        eta_j4 = Jet4_4vector.Eta() 

        # ------- Azimuthal angle -------
        phi_l1 = LEP_1.Phi()
        phi_l2 = LEP_2.Phi()
        phi_j1 = Jet1_4vector.Phi()
        phi_j2 = Jet2_4vector.Phi()              
        phi_j3 = Jet3_4vector.Phi()
        phi_j4 = Jet4_4vector.Phi()    

        # ------- Reconstructed Z boson -------
        Z_VEC = LEP_1 + LEP_2

        MZ = Z_VEC.M()
        pt_z = Z_VEC.Pt()
        eta_z = Z_VEC.Eta()
        phi_z = Z_VEC.Phi()            

        # ------- Two-jet observables -------

        pt_A2_1 = A2_H_vec.Pt()
        pt_A2_2 = A2_NotH_vec.Pt()
        eta_A2_1 = A2_H_vec.Eta()
        eta_A2_2 = A2_NotH_vec.Eta()
        phi_A2_1 = A2_H_vec.Phi()
        phi_A2_2 = A2_NotH_vec.Phi()

        # ------- A3 scalar observables -------

        A3_vec = Jet1_4vector + Jet2_4vector + Jet3_4vector + Jet4_4vector + LEP_1 + LEP_2

        MA3 = A3_vec.M()
        pt_A3 = A3_vec.Pt()
        eta_A3 = A3_vec.Eta()
        phi_A3 = A3_vec.Phi()            

        #Angular variables between final states
        #Three vectors for everything

        T3_light_jet1 = Jet1_4vector.Vect()
        T3_light_jet2 = Jet2_4vector.Vect()
        T3_light_jet3 = Jet3_4vector.Vect()
        T3_light_jet4 = Jet4_4vector.Vect()            
        T3_lep1 = LEP_1.Vect()
        T3_lep2 = LEP_2.Vect()

        # ------- Cossine of polar angle -------

        cos_j1j2 = np.cos(T3_light_jet1.Angle(T3_light_jet2))
        cos_j1j3 = np.cos(T3_light_jet1.Angle(T3_light_jet3))
        cos_j1j4 = np.cos(T3_light_jet1.Angle(T3_light_jet4))
        cos_j2j3 = np.cos(T3_light_jet2.Angle(T3_light_jet3))
        cos_j2j4 = np.cos(T3_light_jet2.Angle(T3_light_jet4))
        cos_j3j4 = np.cos(T3_light_jet3.Angle(T3_light_jet4))
        ##
        cos_j1lep1 = np.cos(T3_light_jet1.Angle(T3_lep1))
        cos_j1lep2 = np.cos(T3_light_jet1.Angle(T3_lep2))
        cos_j2lep1 = np.cos(T3_light_jet2.Angle(T3_lep1))
        cos_j2lep2 = np.cos(T3_light_jet2.Angle(T3_lep2))
        cos_j3lep1 = np.cos(T3_light_jet3.Angle(T3_lep1))
        cos_j3lep2 = np.cos(T3_light_jet3.Angle(T3_lep2))
        cos_j4lep1 = np.cos(T3_light_jet4.Angle(T3_lep1))
        cos_j4lep2 = np.cos(T3_light_jet4.Angle(T3_lep2))
        ##
        cos_lep1lep2 = np.cos(T3_lep1.Angle(T3_lep2))   

        # ------- Delta R -------          

        DR_j1j2 = Jet1_4vector.DeltaR(Jet2_4vector)
        DR_j1j3 = Jet1_4vector.DeltaR(Jet3_4vector)
        DR_j1j4 = Jet1_4vector.DeltaR(Jet4_4vector)
        DR_j2j3 = Jet2_4vector.DeltaR(Jet3_4vector)
        DR_j2j4 = Jet2_4vector.DeltaR(Jet4_4vector)
        DR_j3j4 = Jet3_4vector.DeltaR(Jet4_4vector)
        ##
        DR_j1lep1 = Jet1_4vector.DeltaR(LEP_1)
        DR_j1lep2 = Jet1_4vector.DeltaR(LEP_2)
        DR_j2lep1 = Jet2_4vector.DeltaR(LEP_1)
        DR_j2lep2 = Jet2_4vector.DeltaR(LEP_2)
        DR_j3lep1 = Jet3_4vector.DeltaR(LEP_1)
        DR_j3lep2 = Jet3_4vector.DeltaR(LEP_2)
        DR_j4lep1 = Jet4_4vector.DeltaR(LEP_1)
        DR_j4lep2 = Jet4_4vector.DeltaR(LEP_2)
        ##
        DR_lep1lep2 = LEP_1.DeltaR(LEP_2)                            

        # ------- Delta Phi -------
        DPhi_j1j2 = Jet1_4vector.DeltaPhi(Jet2_4vector)
        DPhi_j1j3 = Jet1_4vector.DeltaPhi(Jet3_4vector)
        DPhi_j1j4 = Jet1_4vector.DeltaPhi(Jet4_4vector)
        DPhi_j2j3 = Jet2_4vector.DeltaPhi(Jet3_4vector)
        DPhi_j2j4 = Jet2_4vector.DeltaPhi(Jet4_4vector)
        DPhi_j3j4 = Jet3_4vector.DeltaPhi(Jet4_4vector)
        ##
        DPhi_j1lep1 = Jet1_4vector.DeltaPhi(LEP_1)
        DPhi_j1lep2 = Jet1_4vector.DeltaPhi(LEP_2)
        DPhi_j2lep1 = Jet2_4vector.DeltaPhi(LEP_1)
        DPhi_j2lep2 = Jet2_4vector.DeltaPhi(LEP_2)
        DPhi_j3lep1 = Jet3_4vector.DeltaPhi(LEP_1)
        DPhi_j3lep2 = Jet3_4vector.DeltaPhi(LEP_2)
        DPhi_j4lep1 = Jet4_4vector.DeltaPhi(LEP_1)
        DPhi_j4lep2 = Jet4_4vector.DeltaPhi(LEP_2)
        ##
        DPhi_lep1lep2 = LEP_1.DeltaPhi(LEP_2)  

        # ------- Delta Theta -------
        DTheta_j1j2 = Jet1_4vector.Theta() - Jet2_4vector.Theta()
        DTheta_j1j3 = Jet1_4vector.Theta() - Jet3_4vector.Theta()
        DTheta_j1j4 = Jet1_4vector.Theta() - Jet4_4vector.Theta()
        DTheta_j2j3 = Jet2_4vector.Theta() - Jet3_4vector.Theta()
        DTheta_j2j4 = Jet2_4vector.Theta() - Jet4_4vector.Theta()
        DTheta_j3j4 = Jet3_4vector.Theta() - Jet4_4vector.Theta()
        ##
        DTheta_j1lep1 = Jet1_4vector.Theta() - LEP_1.Theta()
        DTheta_j1lep2 = Jet1_4vector.Theta() - LEP_2.Theta()
        DTheta_j2lep1 = Jet2_4vector.Theta() - LEP_1.Theta()
        DTheta_j2lep2 = Jet2_4vector.Theta() - LEP_2.Theta()
        DTheta_j3lep1 = Jet3_4vector.Theta() - LEP_1.Theta()
        DTheta_j3lep2 = Jet3_4vector.Theta() - LEP_2.Theta()
        DTheta_j4lep1 = Jet4_4vector.Theta() - LEP_1.Theta()
        DTheta_j4lep2 = Jet4_4vector.Theta() - LEP_2.Theta()
        ##
        DTheta_lep1lep2 = LEP_1.Theta() - LEP_2.Theta()  

        # ------- Cross-section -------
        event_weight = [eevent.Weight for eevent in event.Event]
        xs_weight = event_weight[0] if len(event_weight) == 1 else 0
        
        
        
        
        
        
        
        
        
        
        
        # ------- Angular distributions between the A's, H2 and Z -------

        # ------- Delta Phi -------
        DPhi_A2A2 = A2_H_vec.DeltaPhi(A2_NotH_vec)
        DPhi_A2H2_1 = A2_H_vec.DeltaPhi(H2_vec)
        DPhi_A2H2_2 = A2_NotH_vec.DeltaPhi(H2_vec)
        DPhi_A2Z0_1 = A2_H_vec.DeltaPhi(Z_VEC)
        DPhi_A2Z0_2 = A2_NotH_vec.DeltaPhi(Z_VEC)
        DPhi_H2Z0 = H2_vec.DeltaPhi(Z_VEC)

        # ------- Delta R -------
        DR_A2A2 = A2_H_vec.DeltaR(A2_NotH_vec)
        DR_A2H2_1 = A2_H_vec.DeltaR(H2_vec)
        DR_A2H2_2 = A2_NotH_vec.DeltaR(H2_vec)
        DR_A2Z0_1 = A2_H_vec.DeltaR(Z_VEC)
        DR_A2Z0_2 = A2_NotH_vec.DeltaR(Z_VEC)
        DR_H2Z0 = H2_vec.DeltaR(Z_VEC)

        # ------- Delta Theta -------
        DTheta_A2A2 = A2_H_vec.Theta() - A2_NotH_vec.Theta()
        DTheta_A2H2_1 = A2_H_vec.Theta() - H2_vec.Theta()
        DTheta_A2H2_2 = A2_NotH_vec.Theta() - H2_vec.Theta()
        DTheta_A2Z0_1 = A2_H_vec.Theta() - Z_VEC.Theta()
        DTheta_A2Z0_2 = A2_NotH_vec.Theta() - Z_VEC.Theta()
        DTheta_H2Z0 = H2_vec.Theta() - Z_VEC.Theta()
        
        # ------- Cossine of polar angle -------
        
        T3_A2_1 = A2_H_vec.Vect()
        T3_A2_2 = A2_NotH_vec.Vect()
        T3_H2 = H2_vec.Vect()
        T3_Z = Z_VEC.Vect()    
        
        cos_A2A2 = np.cos(T3_A2_1.Angle(T3_A2_2))
        cos_A2H2_1 = np.cos(T3_A2_1.Angle(T3_H2))
        cos_A2H2_2 = np.cos(T3_A2_2.Angle(T3_H2))
        cos_A2Z0_1 = np.cos(T3_A2_1.Angle(T3_Z))
        cos_A2Z0_2 = np.cos(T3_A2_2.Angle(T3_Z))
        cos_H2Z0_2 = np.cos(H2_vec.Angle(T3_Z))

        # ------- List of all features -------
        Observables = [M_A2_1, M_A2_2, MH, pt_l1, pt_l2, pt_jet1, pt_jet2, pt_jet3, pt_jet4, E_l1,
                        E_l2, E_j1, E_j2, E_j3, E_j4, eta_l1, eta_l2, eta_j1, eta_j2, eta_j3, eta_j4, phi_l1,
                        phi_l2, phi_j1, phi_j2, phi_j3, phi_j4, MZ, pt_z, eta_z, phi_z, pt_A2_1, pt_A2_2, eta_A2_1, MH,
                        pt_H, eta_H, phi_H, eta_A2_2, phi_A2_1, phi_A2_2, MA3, pt_A3, eta_A3, phi_A3, cos_j1j2, cos_j1j3, cos_j1j4, cos_j2j3,
                        cos_j2j4, cos_j3j4, cos_j1lep1, cos_j1lep2, cos_j2lep1, cos_j2lep2, cos_j3lep1, cos_j3lep2, cos_j4lep1,
                        cos_j4lep2, cos_lep1lep2, DR_j1j2, DR_j1j3, DR_j1j4, DR_j2j3, DR_j2j4, DR_j3j4, DR_j1lep1, DR_j1lep2,
                        DR_j2lep1, DR_j2lep2, DR_j3lep1, DR_j3lep2, DR_j4lep1, DR_j4lep2, DR_lep1lep2, DPhi_j1j2, DPhi_j1j3,
                        DPhi_j1j4, DPhi_j2j3, DPhi_j2j4, DPhi_j3j4, DPhi_j1lep1, DPhi_j1lep2, DPhi_j2lep1, DPhi_j2lep2, DPhi_j3lep1,
                        DPhi_j3lep2, DPhi_j4lep1, DPhi_j4lep2, DPhi_lep1lep2, DTheta_j1j2, DTheta_j1j3, DTheta_j1j4, DTheta_j2j3,
                        DTheta_j2j4, DTheta_j3j4, DTheta_j1lep1, DTheta_j1lep2, DTheta_j2lep1, DTheta_j2lep2, DTheta_j3lep1,
                        DTheta_j3lep2, DTheta_j4lep1, DTheta_j4lep2, DTheta_lep1lep2, xs_weight, 
                        DPhi_A2A2, DPhi_A2H2_1, DPhi_A2H2_2, DPhi_A2Z0_1, DPhi_A2Z0_2, DPhi_H2Z0, DR_A2A2, DR_A2H2_1, DR_A2H2_2,
                        DR_A2Z0_1, DR_A2Z0_2, DR_H2Z0, DTheta_A2A2, DTheta_A2H2_1, DTheta_A2H2_2, DTheta_A2Z0_1, DTheta_A2Z0_2,
                        DTheta_H2Z0, cos_A2A2, cos_A2H2_1, cos_A2H2_2, cos_A2Z0_1, cos_A2Z0_2, cos_H2Z0_2]

        entry = pd.DataFrame([Observables], columns=Features)
        df = df.append(entry)

        ###
        out[survived] = np.array(Observables)
        survived += 1

df.to_csv('/home/felipe/JoaoPino/1-BGL-MLProject/Teste_angular.csv'.format(f=proc[0]),sep=',',index=False)


100%|██████████| 100000/100000 [23:15<00:00, 71.66it/s] 


# Plots angular distributions between $A_2$, $A_2^\prime$, $H_2$ and $\mathrm{Z^0}$

In [7]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pandas as pd

In [8]:
Signal = pd.read_csv('/home/felipe/JoaoPino/1-BGL-MLProject/Teste_angular.csv')

In [9]:
mpl.rcParams['text.usetex']=True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{bm}'
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
mpl.rcParams['text.latex.preamble'] = r'\usepackage{inputenc}'
plt.rcParams.update({'font.size': 95})

In [10]:
#Only angular vars
Vars = ['DPhi_A2A2', 'DPhi_A2H2_1', 'DPhi_A2H2_2', 'DPhi_A2Z0_1', 'DPhi_A2Z0_1', 'DPhi_H2Z0', 'DR_A2A2', 'DR_A2H2_1', 'DR_A2H2_2',
        'DR_A2Z0_1', 'DR_A2Z0_1', 'DR_H2Z0', 'DTheta_A2A2', 'DTheta_A2H2_1', 'DTheta_A2H2_2', 'DTheta_A2Z0_1', 'DTheta_A2Z0_1',
        'DTheta_H2Z0', 'cos_A2A2', 'cos_A2H2_1', 'cos_A2H2_2', 'cos_A2Z0_1', 'cos_A2Z0_2', 'cos_H2Z0_2']
Signal = Signal[Vars]

In [16]:
labels = ['$\\Delta \\Phi (A_{2,1}, A_{2,2})$', '$\\Delta \\Phi (A_{2,1}, H_{2})$', '$\\Delta \\Phi (A_{2,2}, H_{2})$', '$\\Delta \\Phi (A_{2,1}, \\mathrm{Z^0})$',
          '$\\Delta \\Phi (A_{2,2}, \\mathrm{Z^0})$', '$\\Delta \\Phi (H_2, \\mathrm{Z^0})$', '$\\Delta R (A_{2,1}, A_{2,2})$',
          '$\\Delta R (A_{2,1}, H_{2})$', '$\\Delta R (A_{2,2}, H_{2})$', '$\\Delta R (A_{2,1}, \\mathrm{Z^0})$', '$\\Delta R (A_{2,2}, \\mathrm{Z^0})$', '$\\Delta R (H_{2}, \\mathrm{Z^0})$',
          '$\\Delta \\theta (A_{2,1}, A_{2,2})$', '$\\Delta \\theta (A_{2,1}, H_{2})$',
          '$\\Delta \\theta (A_{2,2}, H_{2})$', '$\\Delta \\theta (A_{2,1}, \\mathrm{Z^0})$', '$\\Delta \\theta (A_{2,2}, \\mathrm{Z^0})$',
          '$\\Delta \\theta (H_{2}, \\mathrm{Z^0})$', '$\\cos(\\theta_{A_{2,1}, A_{2,2}})$', '$\\cos(\\theta_{A_{2,1}, H_{2}})$',
          '$\\cos(\\theta_{A_{2,2}, H_{2}})$', '$\\cos(\\theta_{A_{2,1}, \\mathrm{Z^0}})$', '$\\cos(\\theta_{A_{2,2}, \\mathrm{Z^0}})$',
          '$\\cos(\\theta_{H_{2}, \\mathrm{Z^0}})$']

In [17]:
num_cols=4
N=len(labels)+1
fig, axes = plt.subplots(nrows=6,ncols=num_cols,figsize=(120,150))
fig.tight_layout(pad=5.0)
axes = axes.flatten()


for i, feature in enumerate(labels):

    var_signal = Signal.iloc[:,i]

    var_signal.hist(ax=axes[i],bins=30,density=True,edgecolor='black',color='#b00505',histtype="stepfilled",label="Signal")
    axes[i].set_xlabel(feature, labelpad = 10.0)
    axes[i].set_ylabel("NE (arb. units)", labelpad = 50.0)
        
        
fig.suptitle('\\textbf{$A_{2,1}$ comes from $H_2$ and $A_{2,2}$ does not come from $H_2$}',fontsize=180) 
plt.tight_layout()
fig.subplots_adjust(top=0.95)
plt.savefig('/home/felipe/JoaoPino/1-BGL-MLProject/Angular_plots.pdf')
#plt.show()

# Code without Jet matching

In [ ]:
Features = ['pt_l1', 'pt_l2', 'pt_jet1', 'pt_jet2', 'pt_jet3', 'pt_jet4',
            'E_l1', 'E_l2', 'E_j1', 'E_j2', 'E_j3', 'E_j4', 'eta_l1', 'eta_l2',
            'eta_j1', 'eta_j2', 'eta_j3', 'eta_j4', 'phi_l1', 'phi_l2',
            'phi_j1', 'phi_j2', 'phi_j3', 'phi_j4', 'MZ', 'pt_z', 'eta_z',
            'phi_z', 'M_j1j2', 'M_j1j3', 'M_j1j4', 'M_j2j3', 'M_j2j4',
            'M_j3j4', 'pt_j1j2', 'pt_j1j3', 'pt_j1j4', 'pt_j2j3',
            'pt_j2j4', 'pt_j3j4', 'eta_j1j2', 'eta_j1j3', 'eta_j1j4',
            'eta_j2j3', 'eta_j2j4', 'eta_j3j4', 'phi_j1j2', 'phi_j1j3',
            'phi_j1j4', 'phi_j2j3', 'phi_j2j4', 'phi_j3j4' ,'MH2_1',
            'MH2_2', 'MH2_3', 'MH2_4', 'MH2_5', 'MH2_6', 'MA3', 'pt_A3',
            'eta_A3', 'phi_A3', 'cos_j1j2', 'cos_j1j3', 'cos_j1j4',
            'cos_j2j3', 'cos_j2j4', 'cos_j3j4', 'cos_j1lep1',
            'cos_j1lep2', 'cos_j2lep1', 'cos_j2lep2', 'cos_j3lep1',
            'cos_j3lep2', 'cos_j4lep1', 'cos_j4lep2', 'cos_lep1lep2',
            'DR_j1j2', 'DR_j1j3', 'DR_j1j4', 'DR_j2j3', 'DR_j2j4',
            'DR_j3j4', 'DR_j1lep1', 'DR_j1lep2', 'DR_j2lep1', 'DR_j2lep2',
            'DR_j3lep1', 'DR_j3lep2', 'DR_j4lep1', 'DR_j4lep2',
            'DR_lep1lep2', 'DPhi_j1j2', 'DPhi_j1j3', 'DPhi_j1j4',
            'DPhi_j2j3', 'DPhi_j2j4', 'DPhi_j3j4', 'DPhi_j1lep1',
            'DPhi_j1lep2', 'DPhi_j2lep1', 'DPhi_j2lep2', 'DPhi_j3lep1',
            'DPhi_j3lep2', 'DPhi_j4lep1', 'DPhi_j4lep2', 'DPhi_lep1lep2',
            'DTheta_j1j2', 'DTheta_j1j3', 'DTheta_j1j4', 'DTheta_j2j3', 'DTheta_j2j4',
            'DTheta_j3j4', 'DTheta_j1lep1', 'DTheta_j1lep2', 'DTheta_j2lep1',
            'DTheta_j2lep2', 'DTheta_j3lep1', 'DTheta_j3lep2', 'DTheta_j4lep1',
            'DTheta_j4lep2', 'DTheta_lep1lep2', 'xs_weight']

for chain, proc in zip(chain_list,Events.items()):
    
    df = pd.DataFrame(columns=Features)

    for count, event in enumerate(tqdm(chain, total=chain.GetEntries())):

        if count == 5000:
            break
        
        out = np.empty((chain.GetEntries(),len(Features)))
        out.fill(np.nan)
        survived = 0
        
        #Get charged leptons (anti-muons and muons)
        muons_eta = list(filter(lambda Muons: np.abs(Muons.Eta) <= 2.47, event.Muon))
        muons = list(filter(lambda Muons: Muons.PT > 20, muons_eta))
        muons = sorted(muons, key=lambda Muon: Muon.PT, reverse=True)          
        
        list_muon = muons

        #Get charged leptons (anti-electrons and electrons)
        electrons_eta = list(filter(lambda Electrons: np.abs(Electrons.Eta) <= 2.47, event.Electron))
        electrons = list(filter(lambda Electrons: Electrons.PT > 20., electrons_eta))
        electrons = sorted(electrons, key=lambda Electron: Electron.PT, reverse=True)  

        list_electron = electrons
        
        list_charged_lep = electrons + muons

        #Get jets. One list with b-jets and another with the light jets

        jet_eta = list(filter(lambda x: abs(x.Eta) <= 2.5, event.Jet))
        jet_filt = list(filter(lambda x: x.PT>20., jet_eta))
        jet_filt_ord = sorted(jet_filt, key=lambda Jet:Jet.PT, reverse=True)
        list_jets = jet_filt_ord

#         bjets, ljets = [], []
#         nbjets = 0
#         for n in range(len(jet_filt_ord)):
#             if jet_filt_ord[n].BTag == 1:
#                 bjets.append(jet_filt_ord[n])
#                 nbjets += 1
#             else:
#                 ljets.append(jet_filt_ord[n])

#         list_bjets = bjets
#         list_light_jets = ljets
        
#         list_jets = ljets + bjets
        

        #Initialize the 4-vectors:
        LEP_vec_1 = ROOT.TLorentzVector()
        LEP_vec_2 = ROOT.TLorentzVector()        
        LIGHT_JET_vec_1 = ROOT.TLorentzVector()
        LIGHT_JET_vec_2 = ROOT.TLorentzVector()
        LIGHT_JET_vec_3 = ROOT.TLorentzVector()
        LIGHT_JET_vec_4 = ROOT.TLorentzVector()


        #Begin analysis
        if (len(list_jets) > 3 and len(list_charged_lep) > 1):

            ####################
            LEP_vec_1.SetPtEtaPhiE(list_charged_lep[0].P4().Pt(),list_charged_lep[0].Eta,
                                   list_charged_lep[0].Phi,list_charged_lep[0].P4().E())
            ####################
            LEP_vec_2.SetPtEtaPhiE(list_charged_lep[1].P4().Pt(),list_charged_lep[1].Eta,
                                   list_charged_lep[1].Phi,list_charged_lep[1].P4().E())
            ####################
            LIGHT_JET_vec_1.SetPtEtaPhiE(list_jets[0].P4().Pt(),list_jets[0].Eta,
                                         list_jets[0].Phi,list_jets[0].P4().E())
            ####################
            LIGHT_JET_vec_2.SetPtEtaPhiE(list_jets[1].P4().Pt(),list_jets[1].Eta,
                                         list_jets[1].Phi,list_jets[1].P4().E())
            ####################
            LIGHT_JET_vec_3.SetPtEtaPhiE(list_jets[2].P4().Pt(),list_jets[2].Eta,
                                         list_jets[2].Phi,list_jets[2].P4().E())
            ####################
            LIGHT_JET_vec_4.SetPtEtaPhiE(list_jets[3].P4().Pt(),list_jets[3].Eta,
                                         list_jets[3].Phi,list_jets[3].P4().E())



            #High-level observables from final states 
                    
            # ------- Transverse momentum -------
            pt_l1 = LEP_vec_1.Pt()
            pt_l2 = LEP_vec_2.Pt()
            pt_jet1 = LIGHT_JET_vec_1.Pt()
            pt_jet2 = LIGHT_JET_vec_2.Pt()
            pt_jet3 = LIGHT_JET_vec_3.Pt()
            pt_jet4 = LIGHT_JET_vec_4.Pt()

            
            # ------- Energy -------
            E_l1 = LEP_vec_1.E()
            E_l2 = LEP_vec_2.E()
            E_j1 = LIGHT_JET_vec_1.E()
            E_j2 = LIGHT_JET_vec_2.E()            
            E_j3 = LIGHT_JET_vec_3.E()
            E_j4 = LIGHT_JET_vec_4.E() 
            
            # ------- Pseudorapidity -------
            eta_l1 = LEP_vec_1.Eta()
            eta_l2 = LEP_vec_2.Eta()
            eta_j1 = LIGHT_JET_vec_1.Eta()
            eta_j2 = LIGHT_JET_vec_2.Eta()          
            eta_j3 = LIGHT_JET_vec_3.Eta()
            eta_j4 = LIGHT_JET_vec_4.Eta() 
            
            # ------- Azimuthal angle -------
            phi_l1 = LEP_vec_1.Phi()
            phi_l2 = LEP_vec_2.Phi()
            phi_j1 = LIGHT_JET_vec_1.Phi()
            phi_j2 = LIGHT_JET_vec_2.Phi()              
            phi_j3 = LIGHT_JET_vec_3.Phi()
            phi_j4 = LIGHT_JET_vec_4.Phi()    
            
            # ------- Reconstructed Z boson -------
            Z_VEC = LEP_vec_1 + LEP_vec_2
            
            MZ = Z_VEC.M()
            pt_z = Z_VEC.Pt()
            eta_z = Z_VEC.Eta()
            phi_z = Z_VEC.Phi()            

            # ------- Two-jet observables -------
            J1J2_VEC = LIGHT_JET_vec_1 + LIGHT_JET_vec_2
            J1J3_VEC = LIGHT_JET_vec_1 + LIGHT_JET_vec_3
            J1J4_VEC = LIGHT_JET_vec_1 + LIGHT_JET_vec_4
            J2J3_VEC = LIGHT_JET_vec_2 + LIGHT_JET_vec_3
            J2J4_VEC = LIGHT_JET_vec_2 + LIGHT_JET_vec_4
            J3J4_VEC = LIGHT_JET_vec_3 + LIGHT_JET_vec_4
            
            
            M_j1j2 = J1J2_VEC.M()
            M_j1j3 = J1J3_VEC.M()
            M_j1j4 = J1J4_VEC.M()
            M_j2j3 = J2J3_VEC.M()
            M_j2j4 = J2J4_VEC.M()
            M_j3j4 = J3J4_VEC.M()
            
            pt_j1j2 = J1J2_VEC.Pt()
            pt_j1j3 = J1J3_VEC.Pt()
            pt_j1j4 = J1J4_VEC.Pt()
            pt_j2j3 = J2J3_VEC.Pt()
            pt_j2j4 = J2J4_VEC.Pt()
            pt_j3j4 = J3J4_VEC.Pt()
            
            eta_j1j2 = J1J2_VEC.Eta()
            eta_j1j3 = J1J3_VEC.Eta()
            eta_j1j4 = J1J4_VEC.Eta()
            eta_j2j3 = J2J3_VEC.Eta()
            eta_j2j4 = J2J4_VEC.Eta()
            eta_j3j4 = J3J4_VEC.Eta() 
            
            phi_j1j2 = J1J2_VEC.Phi()
            phi_j1j3 = J1J3_VEC.Phi()
            phi_j1j4 = J1J4_VEC.Phi()
            phi_j2j3 = J2J3_VEC.Phi()
            phi_j2j4 = J2J4_VEC.Phi()
            phi_j3j4 = J3J4_VEC.Phi()
                                    
            # ------- H2 mass observables -------
            
            H2_VEC_1 = LIGHT_JET_vec_1 + LIGHT_JET_vec_2 + Z_VEC
            H2_VEC_2 = LIGHT_JET_vec_1 + LIGHT_JET_vec_3 + Z_VEC
            H2_VEC_3 = LIGHT_JET_vec_1 + LIGHT_JET_vec_4 + Z_VEC
            H2_VEC_4 = LIGHT_JET_vec_2 + LIGHT_JET_vec_3 + Z_VEC
            H2_VEC_5 = LIGHT_JET_vec_2 + LIGHT_JET_vec_4 + Z_VEC
            H2_VEC_6 = LIGHT_JET_vec_3 + LIGHT_JET_vec_4 + Z_VEC            

            MH2_1 = H2_VEC_1.M()
            MH2_2 = H2_VEC_2.M()            
            MH2_3 = H2_VEC_3.M()            
            MH2_4 = H2_VEC_4.M()            
            MH2_5 = H2_VEC_5.M()
            MH2_6 = H2_VEC_6.M()

            # ------- A3 scalar observables -------

            A3_vec = Z_VEC + LIGHT_JET_vec_1 + LIGHT_JET_vec_2 + LIGHT_JET_vec_3 + LIGHT_JET_vec_4
            
            MA3 = A3_vec.M()
            pt_A3 = A3_vec.Pt()
            eta_A3 = A3_vec.Eta()
            phi_A3 = A3_vec.Phi()            
     
            #Angular variables between final states
            #Three vectors for everything
            
            T3_light_jet1 = LIGHT_JET_vec_1.Vect()
            T3_light_jet2 = LIGHT_JET_vec_2.Vect()
            T3_light_jet3 = LIGHT_JET_vec_3.Vect()
            T3_light_jet4 = LIGHT_JET_vec_4.Vect()            
            T3_lep1 = LEP_vec_1.Vect()
            T3_lep2 = LEP_vec_2.Vect()

            # ------- Cossine of polar angle -------
            
            cos_j1j2 = np.cos(T3_light_jet1.Angle(T3_light_jet2))
            cos_j1j3 = np.cos(T3_light_jet1.Angle(T3_light_jet3))
            cos_j1j4 = np.cos(T3_light_jet1.Angle(T3_light_jet4))
            cos_j2j3 = np.cos(T3_light_jet2.Angle(T3_light_jet3))
            cos_j2j4 = np.cos(T3_light_jet2.Angle(T3_light_jet4))
            cos_j3j4 = np.cos(T3_light_jet3.Angle(T3_light_jet4))
            ##
            cos_j1lep1 = np.cos(T3_light_jet1.Angle(T3_lep1))
            cos_j1lep2 = np.cos(T3_light_jet1.Angle(T3_lep2))
            cos_j2lep1 = np.cos(T3_light_jet2.Angle(T3_lep1))
            cos_j2lep2 = np.cos(T3_light_jet2.Angle(T3_lep2))
            cos_j3lep1 = np.cos(T3_light_jet3.Angle(T3_lep1))
            cos_j3lep2 = np.cos(T3_light_jet3.Angle(T3_lep2))
            cos_j4lep1 = np.cos(T3_light_jet4.Angle(T3_lep1))
            cos_j4lep2 = np.cos(T3_light_jet4.Angle(T3_lep2))
            ##
            cos_lep1lep2 = np.cos(T3_lep1.Angle(T3_lep2))   

            # ------- Delta R -------          
            
            DR_j1j2 = LIGHT_JET_vec_1.DeltaR(LIGHT_JET_vec_2)
            DR_j1j3 = LIGHT_JET_vec_1.DeltaR(LIGHT_JET_vec_3)
            DR_j1j4 = LIGHT_JET_vec_1.DeltaR(LIGHT_JET_vec_4)
            DR_j2j3 = LIGHT_JET_vec_2.DeltaR(LIGHT_JET_vec_3)
            DR_j2j4 = LIGHT_JET_vec_2.DeltaR(LIGHT_JET_vec_4)
            DR_j3j4 = LIGHT_JET_vec_3.DeltaR(LIGHT_JET_vec_4)
            ##
            DR_j1lep1 = LIGHT_JET_vec_1.DeltaR(LEP_vec_1)
            DR_j1lep2 = LIGHT_JET_vec_1.DeltaR(LEP_vec_2)
            DR_j2lep1 = LIGHT_JET_vec_2.DeltaR(LEP_vec_1)
            DR_j2lep2 = LIGHT_JET_vec_2.DeltaR(LEP_vec_2)
            DR_j3lep1 = LIGHT_JET_vec_3.DeltaR(LEP_vec_1)
            DR_j3lep2 = LIGHT_JET_vec_3.DeltaR(LEP_vec_2)
            DR_j4lep1 = LIGHT_JET_vec_4.DeltaR(LEP_vec_1)
            DR_j4lep2 = LIGHT_JET_vec_4.DeltaR(LEP_vec_2)
            ##
            DR_lep1lep2 = LEP_vec_1.DeltaR(LEP_vec_2)                            
            
            # ------- Delta Phi -------
            DPhi_j1j2 = LIGHT_JET_vec_1.DeltaPhi(LIGHT_JET_vec_2)
            DPhi_j1j3 = LIGHT_JET_vec_1.DeltaPhi(LIGHT_JET_vec_3)
            DPhi_j1j4 = LIGHT_JET_vec_1.DeltaPhi(LIGHT_JET_vec_4)
            DPhi_j2j3 = LIGHT_JET_vec_2.DeltaPhi(LIGHT_JET_vec_3)
            DPhi_j2j4 = LIGHT_JET_vec_2.DeltaPhi(LIGHT_JET_vec_4)
            DPhi_j3j4 = LIGHT_JET_vec_3.DeltaPhi(LIGHT_JET_vec_4)
            ##
            DPhi_j1lep1 = LIGHT_JET_vec_1.DeltaPhi(LEP_vec_1)
            DPhi_j1lep2 = LIGHT_JET_vec_1.DeltaPhi(LEP_vec_2)
            DPhi_j2lep1 = LIGHT_JET_vec_2.DeltaPhi(LEP_vec_1)
            DPhi_j2lep2 = LIGHT_JET_vec_2.DeltaPhi(LEP_vec_2)
            DPhi_j3lep1 = LIGHT_JET_vec_3.DeltaPhi(LEP_vec_1)
            DPhi_j3lep2 = LIGHT_JET_vec_3.DeltaPhi(LEP_vec_2)
            DPhi_j4lep1 = LIGHT_JET_vec_4.DeltaPhi(LEP_vec_1)
            DPhi_j4lep2 = LIGHT_JET_vec_4.DeltaPhi(LEP_vec_2)
            ##
            DPhi_lep1lep2 = LEP_vec_1.DeltaPhi(LEP_vec_2)  
            
            # ------- Delta Theta -------
            DTheta_j1j2 = LIGHT_JET_vec_1.Theta() -  LIGHT_JET_vec_2.Theta()
            DTheta_j1j3 = LIGHT_JET_vec_1.Theta() - LIGHT_JET_vec_3.Theta()
            DTheta_j1j4 = LIGHT_JET_vec_1.Theta() - LIGHT_JET_vec_4.Theta()
            DTheta_j2j3 = LIGHT_JET_vec_2.Theta() - LIGHT_JET_vec_3.Theta()
            DTheta_j2j4 = LIGHT_JET_vec_2.Theta() - LIGHT_JET_vec_4.Theta()
            DTheta_j3j4 = LIGHT_JET_vec_3.Theta() - LIGHT_JET_vec_4.Theta()
            ##
            DTheta_j1lep1 = LIGHT_JET_vec_1.Theta() - LEP_vec_1.Theta()
            DTheta_j1lep2 = LIGHT_JET_vec_1.Theta() - LEP_vec_2.Theta()
            DTheta_j2lep1 = LIGHT_JET_vec_2.Theta() - LEP_vec_1.Theta()
            DTheta_j2lep2 = LIGHT_JET_vec_2.Theta() - LEP_vec_2.Theta()
            DTheta_j3lep1 = LIGHT_JET_vec_3.Theta() - LEP_vec_1.Theta()
            DTheta_j3lep2 = LIGHT_JET_vec_3.Theta() - LEP_vec_2.Theta()
            DTheta_j4lep1 = LIGHT_JET_vec_4.Theta() - LEP_vec_1.Theta()
            DTheta_j4lep2 = LIGHT_JET_vec_4.Theta() - LEP_vec_2.Theta()
            ##
            DTheta_lep1lep2 = LEP_vec_1.Theta() - LEP_vec_2.Theta()  

            # ------- Cross-section -------
            event_weight = [eevent.Weight for eevent in event.Event]
            xs_weight = event_weight[0] if len(event_weight) == 1 else 0
            

            # ------- List of all features -------
            Observables = [pt_l1, pt_l2, pt_jet1, pt_jet2, pt_jet3, pt_jet4,
                           E_l1, E_l2, E_j1, E_j2, E_j3, E_j4, eta_l1, eta_l2,
                           eta_j1, eta_j2, eta_j3, eta_j4, phi_l1, phi_l2,
                           phi_j1, phi_j2, phi_j3, phi_j4, MZ, pt_z, eta_z,
                           phi_z, M_j1j2, M_j1j3, M_j1j4, M_j2j3, M_j2j4,
                           M_j3j4, pt_j1j2, pt_j1j3, pt_j1j4, pt_j2j3,
                           pt_j2j4, pt_j3j4, eta_j1j2, eta_j1j3, eta_j1j4,
                           eta_j2j3, eta_j2j4, eta_j3j4, phi_j1j2, phi_j1j3,
                           phi_j1j4, phi_j2j3, phi_j2j4, phi_j3j4 ,MH2_1,
                           MH2_2, MH2_3, MH2_4, MH2_5, MH2_6, MA3, pt_A3,
                           eta_A3, phi_A3, cos_j1j2, cos_j1j3, cos_j1j4,
                           cos_j2j3, cos_j2j4, cos_j3j4, cos_j1lep1,
                           cos_j1lep2, cos_j2lep1, cos_j2lep2, cos_j3lep1,
                           cos_j3lep2, cos_j4lep1, cos_j4lep2, cos_lep1lep2,
                           DR_j1j2, DR_j1j3, DR_j1j4, DR_j2j3, DR_j2j4,
                           DR_j3j4, DR_j1lep1, DR_j1lep2, DR_j2lep1, DR_j2lep2,
                           DR_j3lep1, DR_j3lep2, DR_j4lep1, DR_j4lep2,
                           DR_lep1lep2, DPhi_j1j2, DPhi_j1j3, DPhi_j1j4,
                           DPhi_j2j3, DPhi_j2j4, DPhi_j3j4, DPhi_j1lep1,
                           DPhi_j1lep2, DPhi_j2lep1, DPhi_j2lep2, DPhi_j3lep1,
                           DPhi_j3lep2, DPhi_j4lep1, DPhi_j4lep2, DPhi_lep1lep2, DTheta_j1j2,
                           DTheta_j1j3, DTheta_j1j4, DTheta_j2j3, DTheta_j2j4,
                           DTheta_j3j4, DTheta_j1lep1, DTheta_j1lep2, DTheta_j2lep1,
                           DTheta_j2lep2, DTheta_j3lep1, DTheta_j3lep2, DTheta_j4lep1,
                           DTheta_j4lep2, DTheta_lep1lep2, xs_weight]

            entry = pd.DataFrame([Observables], columns=Features)
            df = df.append(entry)

            ###
            out[survived] = np.array(Observables)
            survived += 1
            
            df.to_csv('/home/joao/Desktop/Root-analysis/1-BGL-MLProject/Data/{f}.csv'.format(f=proc[0]),sep=',',index=False)

# Plots

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pandas as pd

In [ ]:
Signal = pd.read_csv('/home/joao/Desktop/Root-analysis/1-BGL-MLProject/Data/A3_single_prod_160GeV.csv')
Z_plus_jets = pd.read_csv('/home/joao/Desktop/Root-analysis/1-BGL-MLProject/Data/Z_plus_jets.csv')

In [ ]:
mpl.rcParams['text.usetex']=True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{bm}'
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
mpl.rcParams['text.latex.preamble'] = r'\usepackage{inputenc}'
plt.rcParams.update({'font.size': 90})

In [ ]:
#Drop variables
Vars = ['xs_weight']
Signal = Signal.drop(Vars,axis=1)
Z_plus_jets = Z_plus_jets.drop(Vars,axis=1)

In [ ]:
labels = ['$p_T(\\ell_1)$ (GeV)', '$p_T(\\ell_2)$ (GeV)', '$p_T(j_1)$ (GeV)', '$p_T(j_2)$ (GeV)', '$p_T(j_3)$ (GeV)', '$p_T(j_4)$ (GeV)',
            '$E(\\ell_1)$ (GeV)', '$E(\\ell_2)$ (GeV)', '$E(j_1)$ (GeV)', '$E(j_2)$ (GeV)', '$E(j_3)$ (GeV)', '$E(j_4)$ (GeV)', '$\\eta(\\ell_1)$', '$\\eta(\\ell_2)$',
            '$\\eta(j_1)$', '$\\eta(j_2)$', '$\\eta(j_3)$', '$\\eta(j_4)$', '$\\phi(\\ell_1)$', '$\\phi(\\ell_2)$',
            '$\\phi(j_1)$', '$\\phi(j_2)$', '$\\phi(j_3)$', '$\\phi(j_4)$', '$M_{\\mathrm{Z^0}}$ (GeV)', '$p_T(\\mathrm{Z^0})$ (GeV)', '$\\eta(\\mathrm{Z^0})$',
            '$\\phi(\\mathrm{Z^0})$', '$M_{j_1 j_2}$ (GeV)', '$M_{j_1 j_3}$ (GeV)', '$M_{j_1 j_4}$ (GeV)', '$M_{j_2 j_3}$ (GeV)', '$M_{j_2 j_4}$ (GeV)',
            '$M_{j_3 j_4}$ (GeV)', '$p_T(j_1 j_2)$ (GeV)', '$p_T(j_1 j_3)$ (GeV)', '$p_T(j_1 j_4)$ (GeV)', '$p_T(j_2 j_3)$ (GeV)',
            '$p_T(j_2 j_4)$ (GeV)', '$p_T(j_3 j_4)$ (GeV)', '$\\eta(j_1 j_2)$', '$\\eta(j_1 j_3)$', '$\\eta(j_1 j_4)$',
            '$\\eta(j_2 j_3)$', '$\\eta(j_2 j_4)$', '$\\eta(j_3 j_4)$', '$\\phi(j_1 j_2)$', '$\\phi(j_1 j_3)$',
            '$\\phi(j_1 j_4)$', '$\\phi(j_2 j_3)$', '$\\phi(j_2 j_4)$', '$\\phi(j_3 j_4)$' ,'$M(H_{2,1})$ (GeV)',
            '$M(H_{2,2})$ (GeV)', '$M(H_{2,3})$ (GeV)', '$M(H_{2,4})$ (GeV)', '$M(H_{2,5})$ (GeV)', '$M(H_{2,6})$ (GeV)', '$M(A_3)$ (GeV)', '$p_T(A_3)$ (GeV)',
            '$\\eta(A_3)$', '$\\phi(A_3)$', '$\cos(\\theta_{j_1 j_2})$', '$\cos(\\theta_{j_1 j_3})$', '$\cos(\\theta_{j_1 j_4})$',
            '$\cos(\\theta_{j_2 j_3})$', '$\cos(\\theta_{j_2 j_4})$', '$\cos(\\theta_{j_3 j_4})$', '$\cos(\\theta_{j_1 \\ell_1})$',
            '$\cos(\\theta_{j_1 \\ell_2})$', '$\cos(\\theta_{j_2 \\ell_1})$', '$\cos(\\theta_{j_2 \\ell_2})$', '$\cos(\\theta_{j_3 \\ell_1})$',
            '$\cos(\\theta_{j_3 \\ell_2})$', '$\cos(\\theta_{j_4 \\ell_1})$', '$\cos(\\theta_{j_4 \\ell_2})$', '$\cos(\\theta_{\\ell_1 \\ell_2})$',
            '$\\Delta R(j_1,j_2)$', '$\\Delta R(j_1,j_3)$', '$\\Delta R(j_1,j_4)$', '$\\Delta R(j_2,j_3)$', '$\\Delta R(j_2,j_4)$',
            '$\\Delta R(j_3,j_4)$', '$\\Delta R(j_1,\\ell_1)$', '$\\Delta R(j_1,\\ell_2)$', '$\\Delta R(j_2,\\ell_1)$', '$\\Delta R(j_2,\\ell_2)$',
            '$\\Delta R(j_3,\\ell_1)$', '$\\Delta R(j_3,\\ell_2)$', '$\\Delta R(j_4,\\ell_1)$', '$\\Delta R(j_4,\\ell_2)$',
            '$\\Delta R(\\ell_1,\\ell_2)$', '$\\Delta \\phi(j_1,j_2)$', '$\\Delta \\phi(j_1,j_3)$', '$\\Delta \\phi(j_1,j_4)$',
            '$\\Delta \\phi(j_2,j_3)$', '$\\Delta \\phi(j_2,j_4)$', '$\\Delta \\phi(j_3,j_4)$', '$\\Delta \\phi(j_1,\\ell_1)$',
            '$\\Delta \\phi(j_1,\\ell_2)$', '$\\Delta \\phi(j_2,\\ell_2)$', '$\\Delta \\phi(j_2,\\ell_2)$', '$\\Delta \\phi(j_3,\\ell_1)$',
            '$\\Delta \\phi(j_3,\\ell_2)$', '$\\Delta \\phi(j_4,\\ell_1)$', '$\\Delta \\phi(j_4,\\ell_2)$', '$\\Delta \\phi(\\ell_1,\\ell_2)$',
            '$\\Delta \\theta(j_1,j_2)$', '$\\Delta \\theta(j_1,j_3)$', '$\\Delta \\theta(j_1,j_4)$', '$\\Delta \\theta(j_2,j_3)$', '$\\Delta \\theta(j_2,j_4)$',
            '$\\Delta \\theta(j_3,j_4)$', '$\\Delta \\theta(j_1,\\ell_1)$', '$\\Delta \\theta(j_1,\\ell_2)$', '$\\Delta \\theta(j_2,\\ell_1)$',
            '$\\Delta \\theta(j_2,\\ell_2)$', '$\\Delta \\theta(j_3,\\ell_1)$', '$\\Delta \\theta(j_3,\\ell_2)$', '$\\Delta \\theta(j_4,\\ell_1)$',
            '$\\Delta \\theta(j_4,\\ell_2)$', '$\\Delta \\theta(\\ell_1,\\ell_2)$']

In [ ]:
num_cols=4
N=len(labels)+1
fig, axes = plt.subplots(nrows=31,ncols=num_cols,figsize=(120,350))
fig.tight_layout(pad=5.0)
axes = axes.flatten()


for i, feature in enumerate(labels):

    var_signal = Signal.iloc[:,i]
    var_z_plus_jets = Z_plus_jets.iloc[:,i]

    var_signal.hist(ax=axes[i],bins=30,density=True,edgecolor='black',color='#b00505',histtype="stepfilled",label="Signal")
    var_z_plus_jets.hist(ax=axes[i],bins=30,density=True,edgecolor='#0216f0',color="#0216f0",histtype="step",label="$\\mathrm{Z^0+jets}$", linewidth=6.0)
    axes[i].set_xlabel(feature, labelpad = 10.0)
    axes[i].set_ylabel("NE (arb. units)", labelpad = 50.0)
        
        
fig.suptitle('\\textbf{All observables}',fontsize=180) 
plt.tight_layout()
fig.subplots_adjust(top=0.97)
plt.savefig('//home/joao/Desktop/Root-analysis/1-BGL-MLProject/Plots/All_plots_159GeV.pdf')
#plt.show()

In [ ]:
num_cols=3
N=len(labels)+1
fig, axes = plt.subplots(nrows=3,ncols=num_cols,figsize=(120,70))
fig.tight_layout(pad=5.0)
axes = axes.flatten()

for i, feature in enumerate(labels):

    var_signal = Signal.iloc[:,i]
    var_z_plus_jets = Z_plus_jets.iloc[:,i]
    if (feature == '$M_{j_1, j_2}$ (GeV)' or feature == '$M_{j_1, j_3}$ (GeV)' or feature == '$M_{j_1, j_4}$ (GeV)' or
       feature == '$M_{j_2, j_3}$ (GeV)' or feature == '$M_{j_3, j_4}$ (GeV)' or feature == '$M_{j_2, j_4}$ (GeV)'):
    
        var_signal.hist(ax=axes[i],range=(0,1000),bins=30,density=True,edgecolor='black',color='#b00505',histtype="stepfilled",label="Signal")
        var_z_plus_jets.hist(ax=axes[i],range=(0,1000),bins=30,density=True,edgecolor='#0216f0',color="#0216f0",histtype="step",label="$\\mathrm{Z^0+jets}$", linewidth=6.0)
        axes[i].set_xlabel(feature, labelpad = 10.0)
        axes[i].set_ylabel("NE (arb. units)", labelpad = 50.0)
    
    else:
        
        var_signal.hist(ax=axes[i],bins=30,density=True,edgecolor='black',color='#b00505',histtype="stepfilled",label="Signal")
        var_z_plus_jets.hist(ax=axes[i],bins=30,density=True,edgecolor='#0216f0',color="#0216f0",histtype="step",label="$\\mathrm{Z^0+jets}$", linewidth=6.0)
        axes[i].set_xlabel(feature, labelpad = 10.0)
        axes[i].set_ylabel("NE (arb. units)", labelpad = 50.0)
        
        
fig.suptitle('\\textbf{Mass observables for signal and z+jets}',fontsize=180) 
plt.tight_layout()
fig.subplots_adjust(top=0.91)
plt.savefig('//home/joao/Desktop/Root-analysis/1-BGL-MLProject/Plots/Mass_plots.pdf')
#plt.show()